In [304]:
import json
import sys
import re
from bs4 import BeautifulSoup
import os
from src.util import count_lines_many, data_from_many, bunch_paragraphs, unpack
import numpy as np

FN = 'nydailynews'

DATAPATH = '/datapool/news_articles/raw_data/'+FN+'/'
MAX_PER_FILE = 1000

datafiles = [DATAPATH + file for file in os.listdir(DATAPATH)]

"""PREPROCESS AND PARAGRAPH EXTRACTION"""

def preprocess(item):
    url, html = unpack(item)
    label = 'none'#get_label(url,html)
    paragraphs = get_paragraphs(html)
    return label, paragraphs


def get_paragraphs(html):
    soup = BeautifulSoup(html,'html.parser')
    paragraphs = [
            tag.get_text() for tag in soup.findAll('title')]+[
                    item['content'] for item in soup.findAll('meta',property=True) if item['property'] == 'og:description']+[
                            tag.get_text() for tag in soup.findAll('p')]
    if len(paragraphs) != 0:
        return paragraphs
    else:
        print("NOCONTENT")
        return 'NOCONTENT'
    
    
data = map(preprocess,data_from_many(datafiles))

In [361]:
def filter_paragraphs(FN,paragraphs):
    filtered_paragraphs = []
    
    if FN == 'nytimes':
        forbidden = 'New York Times,|,not a robot,Invalid email address,feedback on this page,Home Page,@nytimes.com,NYTimes.com,contributed reporting.,newsletter.,updates via Facebook'.split(',')
    elif FN == 'wsj':
        forbidden = 'WSJ,All Rights Reserved,The Wall Street Journal,summary.headline,contributed to this article,Photo: ,\n and ,sign in'.split(',')
    elif FN == 'wapo':
        forbidden = 'The Washington Post,Show me more offers,related: ,[,],sign in.,sign up,check your inbox,signed up,you might also like, AM, PM,valid email address,This post has been updated,|'.split(',')
    elif FN == 'nypost':
        forbidden = '|,privacy policy.,other great newsletters,…,information services.,delivered to your inbox,received your submission.,\t\t\t\n'.split(',')
    elif FN == 'nydailynews':
        forbidden = 'Copyright ©,New York Daily News,on a mobile device?,Get our instant notifications as news happens,You can manage them anytime using browser settings,Sign up now to start receiving breaking news alerts on your desktop'.split(',')
    
    for paragraph in paragraphs:
        res = True
        
        if len(paragraph.split(' ')) <= 4:
            res = False
        else:
            pass
        
        for item in forbidden:
            if item.lower() in paragraph.lower():
                res = False
            else:
                pass
                
        if paragraph[:3] == 'By ':
            res = False
        else:
            pass
            
        if paragraph[:7] == 'Follow ':
            res = False
        else:
            pass
        
        if paragraph.isupper():
            res = False
        else:
            pass
        
        if res:
            filtered_paragraphs.append(paragraph)
                
    return filtered_paragraphs[1:]

In [362]:
label, paragraphs = next(data)

In [363]:
filter_paragraphs('nydailynews',paragraphs)

['Raise your glasses, ladies and gentlemen of New York, and toast the jury that convicted Sheldon Silver of corruption after...',
 "Raise your glasses, ladies and gentlemen of New York, and toast the jury that convicted Sheldon Silver of corruption after stunningly brief deliberations. The label of felon is affixed to the once-mighty man in a citizens' exercise of truth-telling that is sweet and joyous and, in its limitations, sorrowful.",
 "The jury's verdict against the former Assembly speaker rid New York's political culture of its most virulent and resistant bacterium. For, until his indictment, Silver had long been a pathogen who infected governing with anti-democratic abuses.",
 'There is satisfaction in knowing that the big, fake tough guy will likely live behind bars for a substantial portion of the time he has left before senescence and frailty set in.',
 'Silver deserves every second of the powerlessness that is a torment of imprisonment, along with the pain of searing regret